In [7]:
import pandas as pd
import pandas as pd
import numpy as np
# 기존 plotnine(ggplot)코드를 pandas와 seaborn으로 변경
# from plotnine import *
import seaborn as sns
# 지도 시각화를 위해
import folium

import warnings
warnings.filterwarnings('ignore')
# warning 메시지 제거, warning 메시지는 라이브러리 업데이트나 사용법에 대한 안내 등이 있습니다.
# 코딩을 처음 시작할 때는  warning 메시지가 나오면 당황하실 수도 있어서 제거를 하고 보도록 합니다.
# warning 메시지는 제거하고 보셔도 되지만 Error 메시지를 꼭 고쳐주셔야 합니다.
warnings.filterwarnings('ignore', 'This pattern has match groups')
warnings.filterwarnings('ignore', 'The iterable function was deprecated in Matplotlib')

import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
# Window 의 한글 폰트 설정
# plt.rc('font',family='Malgun Gothic')
# Mac 의 한글 폰트 설정
plt.rc('font',family='Malgun Gothic') 
plt.rc('axes', unicode_minus=False)

set_matplotlib_formats('retina')

import json
import mpld3

In [8]:
def make_table(file,col):
    path='D:\\사용자\\USER\\Desktop\\데이터\\인천\\'+file+'.csv'
    
    df=pd.read_csv(path, encoding='cp949')
    df['구']=''
    name_split = df["도로명전체주소"].str.split(" ")
    df["구"] = name_split.str.get(1)
    df = df.drop(columns=['도로명전체주소','소재지전체주소',], axis=1)
    df=df.groupby('구')
    df=df.count().reset_index()
    df.columns = ['구',col]
    df= df.sort_values(by=col,ascending=False)

    return df

In [20]:
h1=make_table("대형병원","대형병원")
h2=make_table("산후조리원","산후조리원")

h2.loc[len(h2)] = ['중구',0]
h2.loc[len(h2)] = ['동구',0]
h2.loc[len(h2)] = ['강화군',0]
h2.loc[len(h2)] = ['옹진군',0]


h3=make_table("안마원","안마원")
h4=make_table("약국","약국")
h5=make_table("병원","병원")
m1=pd.merge(h1, h2 ,on="구")
m2=pd.merge(h3, h4 ,on="구")
m3=pd.merge(m1, m2 ,on="구")
m4=pd.merge(m3, h5 ,on="구")
m4

,구,대형병원,산후조리원,안마원,약국,병원
0,남동구,45,4,13,233,705
1,서구,38,7,10,171,495
2,미추홀구,36,4,9,177,440
3,부평구,30,2,9,226,607
4,계양구,27,3,5,119,347
5,연수구,17,4,8,114,398
6,중구,11,0,4,60,118
7,강화군,8,0,3,23,80
8,동구,2,0,3,35,86
9,옹진군,1,0,1,4,27


In [19]:
li=['중구','동구','미추홀구','연수구','남동구','부평구',
    '계양구','서구','강화군','옹진군']


li2=list(h5['구'])

for i in li:
    if(i not in li2):
        print(i+" ")

In [21]:
df=pd.read_csv(r'D:\사용자\USER\Desktop\데이터\인천\인천인구.csv', encoding='cp949', thousands=',')
df['구']=''
name_split = df["행정구역"].str.split(" ")
df["구"] = name_split.str.get(1)

df = df.drop(columns=['행정구역'], axis=1)

m4=pd.merge(m4, df ,on="구")
m4

,구,대형병원,산후조리원,안마원,약국,병원,2021년08월_계_총인구수
0,남동구,45,4,13,233,705,519667
1,서구,38,7,10,171,495,544726
2,미추홀구,36,4,9,177,440,405811
3,부평구,30,2,9,226,607,489178
4,계양구,27,3,5,119,347,295492
5,연수구,17,4,8,114,398,390107
6,중구,11,0,4,60,118,141341
7,강화군,8,0,3,23,80,69324
8,동구,2,0,3,35,86,62410
9,옹진군,1,0,1,4,27,20373


In [22]:
m4.to_excel('D:\사용자\\USER\\Desktop\\데이터\\인천\\인천건강.xlsx', index=False)

In [34]:
def make_plot(file,col):
    
    path='D:\\사용자\\USER\\Desktop\\데이터\\인천\\'+file+'.csv'
    
    
    library=pd.read_csv(path, encoding='cp949')

    df=library
    df = pd.DataFrame({'group':df['구'], 'values':df[col] })

    return df

In [35]:
seoul_health1=make_plot("수도권건강",'대형병원')
seoul_health2=make_plot("수도권건강",'산후조리원')
seoul_health3=make_plot("수도권건강",'안마원')
seoul_health4=make_plot("수도권건강",'약국')
seoul_health5=make_plot("수도권건강",'병원')

In [36]:
def normal(df):
    data=list(df['values'])
    std_data = (data - np.mean(data,axis = 0))/ np.std(data,axis = 0)
    normalization=(std_data-min(std_data))/(max(std_data)-min(std_data))
    df['values']=normalization
    return df

In [37]:
d1=normal(seoul_health1)
d2=normal(seoul_health2)
d3=normal(seoul_health3)
d4=normal(seoul_health4)
d5=normal(seoul_health5)
mm1=pd.merge(d1, d2 ,on="group")
mm2=pd.merge(d3, d4 ,on="group")
mm3=pd.merge(mm1, mm2 ,on="group")
mm4=pd.merge(mm3, d5 ,on="group")
mm4

,group,values_x_x,values_y_x,values_x_y,values_y_y,values
0,강남구,1.000000,1.000000,1.000000,0.626756,1.000000
1,송파구,0.414103,0.454802,0.095899,0.342651,0.233027
2,강서구,0.416174,0.520004,0.213165,0.222041,0.161562
3,강동구,0.489003,0.788768,0.000000,0.303300,0.225256
4,동대문구,0.664339,0.591394,0.490610,0.556494,0.219381
...,...,...,...,...,...,...
61,연수구,0.256206,0.341456,0.237172,0.082448,0.057576
62,인천 중구,0.546341,0.000000,0.368750,0.196185,0.014510
63,강화군,0.864652,0.000000,0.621578,0.116453,0.088653
64,동구,0.158515,0.000000,0.702522,0.313382,0.140689


In [38]:
mm4.to_excel('D:\사용자\\USER\\Desktop\\데이터\\인천\\수도권건강정규화.xlsx', index=False)

In [40]:
library=pd.read_csv('D:\\사용자\\USER\\Desktop\\데이터\\인천\\수도권건강정규화.csv', encoding='cp949')
library2=pd.read_csv('D:\\사용자\\USER\\Desktop\\데이터\\인천\\수도권건강행복지도.csv', encoding='cp949')

mmm=pd.merge(library2, library ,on="구")
mmm

,구,정신건강증진기관,의사,의료기관병상,대형병원,산후조리원,안마원,약국,병원
0,종로구,0.6914,1.0000,0.7616,0.469067,0.457587,0.470928,0.924054,0.580323
1,중구,0.4106,0.9995,0.4043,0.645565,0.542134,0.578091,1.000000,0.849296
2,용산구,0.3353,0.6046,0.1433,0.000000,0.148020,0.116074,0.312736,0.148935
3,성동구,0.3321,0.5783,0.2111,0.210360,0.115543,0.300998,0.278678,0.181675
4,광진구,0.7393,0.6352,0.1936,0.184713,0.292539,0.237009,0.208847,0.191310
...,...,...,...,...,...,...,...,...,...
61,계양구,0.2501,0.3963,0.4426,0.661105,0.338092,0.176623,0.177452,0.093372
62,서구,0.1375,0.3737,0.3267,0.478008,0.427936,0.200883,0.101100,0.031668
63,미추홀구,0.2503,0.3817,0.4265,0.638556,0.328243,0.265378,0.206209,0.072448
64,강화군,0.9972,0.3578,0.4366,0.864652,0.000000,0.621578,0.116453,0.088653


In [41]:
mmm.to_excel('D:\사용자\\USER\\Desktop\\데이터\\인천\\수도권건강최종.xlsx', index=False)